# Data preprocessing

---


In [2]:
DATA_PATH = r'..\data\processed\04-final_data_v2.pkl'


DATA_X_TRAIN = r'..\data\processed\X_train.pkl'

DATA_X_TEST = r'..\data\processed\X_test.pkl'

DATA_y_TRAIN = r'..\data\processed\y_train.pkl'

DATA_y_TEST = r'..\data\processed\y_test.pkl'



list_columns = ['Key_languages', 'Databases', 'Web_frames', 'Platforms']

specified_columns = ['EdLevel', 'experanse_years']

## Importing


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

# Set the maximum number of rows and columns to display
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [4]:
# Functions
def encoding_list_col(df):
    # Perform one-hot encoding on list columns

    for column in list_columns:
        # Create a list of unique values from all lists in the column
        unique_values = set(
            value for sublist in df[column] for value in sublist)

        # Create binary features for each unique value
        for value in unique_values:
            df[f"{column}_{value}"] = df[column].apply(
                lambda x: int(value in x))

    # Drop the original list columns
    df = df.drop(columns=list_columns)

    # Rearrange columns to move specified columns to the end

    df = pd.concat([df.drop(columns=specified_columns),
                    df[specified_columns]], axis=1)

    return df

In [5]:
df = pd.read_pickle(DATA_PATH)

### 1)Feature selection and Engineering


In [6]:
df.head()

,Key_languages,Databases,Platforms,Web_frames,EdLevel,experanse_years,DevType
0,"[GDScript, Bash/Shell (all shells), Rust, Java...",[Supabase],"[Amazon Web Services (AWS), Netlify, Vercel]","[Next.js, React, Remix, Vue.js]",Bachelor's,9.0,Executive
1,"[Go, Rust, OCaml, Haskell, Bash/Shell (all she...",[Don't have],"[Amazon Web Services (AWS), Google Cloud, Open...",[Don't have],Bachelor's,23.0,Back-end Developer
2,"[Ruby, JavaScript, TypeScript, SQL, PHP, HTML/...","[PostgreSQL, Redis]","[Cloudflare, Heroku]","[Node.js, React, Ruby on Rails, Vue.js, WordPr...",Bachelor's,7.0,Front-end Developer
3,"[Rust, JavaScript, Python, HTML/CSS, TypeScript]","[BigQuery, Elasticsearch, MongoDB, PostgreSQL]","[Amazon Web Services (AWS), Firebase, Heroku, ...","[Express, Gatsby, NestJS, Next.js, Node.js, Re...",Bachelor's,4.0,Full-stack Developer
4,"[Go, Ruby, Rust, JavaScript, TypeScript, SQL, ...","[BigQuery, Cloud Firestore, PostgreSQL, Redis]","[Amazon Web Services (AWS), Cloudflare, Google...","[Angular, Express, NestJS, Node.js]",Some College/University,21.0,Back-end Developer


In [8]:
df = encoding_list_col(df)

In [9]:
df['DevType'].value_counts()

Full-stack Developer             23198
Back-end Developer               12679
Front-end Developer               4337
Desktop/Enterprise Developer      3615
Mobile Developer                  2302
Engineering Manager               1974
Embedded Developer                1713
Data Scientist/ML Specialist      1377
Executive                         1288
DevOps Specialist                 1276
R&D Developer                     1255
Data Engineer                     1145
Academic Researcher               1051
Cloud Infrastructure Engineer      955
Game/Graphics Developer            719
Data/Business Analyst              670
System Administrator               571
QA/Test Developer                  508
Project Manager                    497
Product Manager                    415
Site Reliability Engineer          402
Security Professional              395
Educator                           325
Scientist                          307
Developer Experience               298
Blockchain Developer     

In [10]:
df = df[~((df['DevType'] == 'Marketing/Sales Professional') | (df['DevType'] == 'Developer Advocate') | (df['DevType'] == 'Designer') | (df['DevType'] == 'Scientist')
          | (df['DevType'] == 'Educator') | (df['DevType'] == 'Product Manager') | (df['DevType'] == 'Project Manager') | (df['DevType'] == 'Developer Experience ') | (df['DevType'] == 'Engineering Manager')
          )]

In [12]:
df['DevType'].value_counts()

Full-stack Developer             23198
Back-end Developer               12679
Front-end Developer               4337
Desktop/Enterprise Developer      3615
Mobile Developer                  2302
Embedded Developer                1713
Data Scientist/ML Specialist      1377
Executive                         1288
DevOps Specialist                 1276
R&D Developer                     1255
Data Engineer                     1145
Academic Researcher               1051
Cloud Infrastructure Engineer      955
Game/Graphics Developer            719
Data/Business Analyst              670
System Administrator               571
QA/Test Developer                  508
Site Reliability Engineer          402
Security Professional              395
Developer Experience               298
Blockchain Developer               253
Hardware Engineer                  237
Database Administrator             213
Name: DevType, dtype: int64

Done


### 2)split X and y


In [14]:
X = df.drop('DevType', axis=1)
y = df['DevType']

### **Balance classes:Resample**


In [15]:
samples_per_class = 1400
y_cont = y.value_counts()

for index, values in y_cont.iteritems():
    if values > samples_per_class:
        under_s = RandomUnderSampler(
            sampling_strategy={index: 1713}, random_state=42)
        X, y = under_s.fit_resample(X, y)

    elif values < samples_per_class:
        under_s = RandomOverSampler(
            sampling_strategy={index: 1377}, random_state=42)
        X, y = under_s.fit_resample(X, y)

In [16]:
y.value_counts()

Embedded Developer               1713
Front-end Developer              1713
Mobile Developer                 1713
Desktop/Enterprise Developer     1713
Full-stack Developer             1713
Back-end Developer               1713
Site Reliability Engineer        1377
Security Professional            1377
R&D Developer                    1377
QA/Test Developer                1377
Hardware Engineer                1377
Game/Graphics Developer          1377
Academic Researcher              1377
Executive                        1377
Developer Experience             1377
DevOps Specialist                1377
Database Administrator           1377
Data/Business Analyst            1377
Data Scientist/ML Specialist     1377
Data Engineer                    1377
Cloud Infrastructure Engineer    1377
Blockchain Developer             1377
System Administrator             1377
Name: DevType, dtype: int64

### 3)Trian test split


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [19]:
y_train.value_counts()

Full-stack Developer             1464
Back-end Developer               1460
Embedded Developer               1459
Mobile Developer                 1450
Desktop/Enterprise Developer     1448
Front-end Developer              1421
Site Reliability Engineer        1193
Database Administrator           1192
Academic Researcher              1186
Data/Business Analyst            1185
Data Engineer                    1182
Cloud Infrastructure Engineer    1177
Executive                        1177
DevOps Specialist                1175
Security Professional            1168
R&D Developer                    1167
System Administrator             1167
Developer Experience             1166
Game/Graphics Developer          1166
Hardware Engineer                1164
QA/Test Developer                1164
Data Scientist/ML Specialist     1157
Blockchain Developer             1145
Name: DevType, dtype: int64

### 4)preprocessing pipeline


In [22]:
num_feat = X_train.select_dtypes(exclude='object').columns
cat_feat = X_train.select_dtypes(include='object').columns

scaler = RobustScaler()
encoder = OneHotEncoder()

In [24]:

num_pipeline = Pipeline(
    steps=[
        ('scaler', scaler)
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(drop='first'))
        # Set with_mean=False for sparse input
        # ('scaler', StandardScaler(with_mean=False)),
    ]
)

preperocessor = ColumnTransformer(
    [
        ('num_pipeline', num_pipeline, num_feat),
        ('cat_pipeline', cat_pipeline, cat_feat)
    ]
)

In [25]:
X_train_arr = preperocessor.fit_transform(X_train)
X_test_arr = preperocessor.transform(X_test)

In [26]:
X_train_arr.shape

(28633, 152)

In [27]:
X_train_df = pd.DataFrame(
    X_train_arr, columns=preperocessor.get_feature_names_out())
X_test_df = pd.DataFrame(
    X_test_arr, columns=preperocessor.get_feature_names_out())

In [28]:
# X_train_df

,num_pipeline__Key_languages_Lua,num_pipeline__Key_languages_Kotlin,num_pipeline__Key_languages_PHP,num_pipeline__Key_languages_Visual Basic (.Net),num_pipeline__Key_languages_Crystal,num_pipeline__Key_languages_Go,num_pipeline__Key_languages_Python,num_pipeline__Key_languages_Cobol,num_pipeline__Key_languages_Nim,num_pipeline__Key_languages_Lisp,...,num_pipeline__Platforms_Vercel,num_pipeline__Platforms_Cloudflare,num_pipeline__experanse_years,cat_pipeline__EdLevel_Bachelor's,cat_pipeline__EdLevel_Master's,cat_pipeline__EdLevel_Other,cat_pipeline__EdLevel_Primary/Elementary,cat_pipeline__EdLevel_Professional,cat_pipeline__EdLevel_Secondary School,cat_pipeline__EdLevel_Some College/University
0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.090909,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.090909,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,...,0.0,1.0,-0.363636,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.545455,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.818182,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.272727,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28629,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.363636,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28630,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.545455,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.363636,0.0,0.0,0.0,0.0,1.0,0.0,0.0


---


### Save data


In [29]:
X_train_df.to_pickle(DATA_X_TRAIN)
X_test_df.to_pickle(DATA_X_TEST)

# pd.to_pickle(X_train_arr , DATA_X_TRAIN)
# pd.to_pickle(X_test_arr , DATA_X_TEST)

# pd.to_pickle(y_train , y_train)
# pd.to_pickle(y_test , DATA_y_TEST)


y_train.to_pickle(DATA_y_TRAIN)
y_test.to_pickle(DATA_y_TEST)

**Conclusion:**
The notebook successfully addresses several crucial steps in the data preprocessing phase, including feature engineering, class balancing, and creating an efficient preprocessing pipeline. It also demonstrates good practices such as stratified sampling and saving intermediate data.
